In [ ]:
import pandas as pd
import numpy as np
import matplotlib as plt

df = pd.read_stata('data/Authority.dta')
print(df.shape)
#list(df.columns)

#construct work category dummy
df['OG03_dummy'] = 0
df.loc[(df['work_category']=='OG03')&(df['work_category']!=''),'OG03_dummy'] = 1

df['OG01_dummy'] = 0
df.loc[(df['work_category']=='OG01')&(df['work_category']!=''),'OG01_dummy'] = 1

df['OG_rest_dummy'] = 0
df.loc[(df['OG01_dummy']!=1)&(df['OG03_dummy']!=1)&(df['work_category']!=''),'OG_rest_dummy'] = 1

df['OG_dummy'] = 0
df.loc[df['work_category'].str[0:2] == 'OG','OG_dummy'] = 1

df['OS_dummy'] = 0
df.loc[df['work_category'].str[0:2] == 'OS','OS_dummy'] = 1

#treated vs controls
df['trend'] = df['year'] - 1999

df['trend_treat'] = df['trend']
df.loc[(df['authority_code']!=3090272)&(df['authority_code']!=3070001),'trend_treat'] = 0
#15225 real change made / no zeros in df['trend']
#print(df['trend_treat'].value_counts()) chekced

df['trend_control'] = df['trend']
df.loc[(df['authority_code']==3090272)|(df['authority_code']==3070001),'trend_control'] = 0
#902 real change made
#print(df['trend_control'].value_counts()) checked

#PA specifics
df = df.sort_values(by='authority_code',ascending=True)
#auth = econ.group_id(df, cols = 'authority_code')
#print(auth) #dataframe

auth_list = df['authority_code'].values.tolist()
auth_list = list(set(auth_list))
#겹치는 부분 = authority_code야
#id_auth = group_id + 1 if df의 auth code == auth의 code

df['id_auth'] = 0
for i in range(len(df)):
    for j in range(len(auth_list)):
        if df.loc[i,'authority_code'] == auth_list[j]:
            df.loc[i,'id_auth'] = j+1


'''for i in range(len(auth_list)):
    name = 'trend_pa_'+str(i+1)
    df[name] = 0
    for j in range(len(df)):
        if df.loc[j, 'id_auth'] == i+1 :
            df.loc[j, name] = df.loc[j,'trend']
        if df.loc[j, 'authority_code']==3090272 or df.loc[j, 'authority_code']==3070001:
            df.loc[j, name] = 0'''
#for statement check            
#df.head
#print(max(df['id_path']))

(16127, 31)


In [ ]:
#check 
print(df.loc[33,'trend_pa_1']) #should be 33 = 1, 34 = 0

In [6]:
#!pip install linearmodels  
from linearmodels import PanelOLS
import statsmodels.api as sm

In [28]:
#2 econtools
!pip install econtools
import econtools as econ
import econtools.metrics as mt

In [4]:
#Iwork 1~42 pattern unfound
#G인지 S인지 판단
#G이면 sorted된 거에서 인덱스 +1 =Iwork_nm
#S이면 sorted된 거에서 index + 17

''''work_ca = df['work_category'].values.tolist()
work_ca = list(set(work_ca))
#print(work_ca)
#total 42

os_list = []
og_list = []
for i in range(len(work_ca)):
    if work_ca[i][1]=='G':
        work_ca[i] = int(work_ca[i][2:])
        og_list.append(work_ca[i])
    else:
        work_ca[i] = int(work_ca[i][2:])
        os_list.append(work_ca[i])

os_list = sorted(os_list)
og_list = sorted(og_list)


print(df.loc[0,'work_category'][1])

work_ca_order = og_list + os_list
#work_ca_order 대신 (1,43)

for i in range(1,43):
    name = 'Iwork_ca_'+str(i+1) 
    df[name] = 0
    for j in range(len(df)):
        if df.loc[j,'work_category'][1] == 'G':
            work_OG = int(df.loc[j, 'work_category'][2:])
            for g in range(len(og_list)):
                if work_OG == og_list[g]:
                    idx_g = str(g+1)
                    df.loc[j, 'Iwork_ca_' + idx_g] = 1
        else:
            work_OS = int(df.loc[j, 'work_category'][2:])
            for s in range(len(os_list)):
                if work_OS == os_list[s]:
                    idx_s = str(s + 17)
                    df.loc[j, 'Iwork_ca_'+ idx_s] = 1'''

S


In [5]:
#2001~2010 dummy
'''for j in range(2001,2011):
    df['_Iyear_'+str(j)] = 0
    for i in range(len(df)):
        if df.loc[i,'year'] == j:
            df.loc[i, '_Iyear_'+str(j)] = 1'''

In [3]:
work_dum = pd.get_dummies(df['work_category'])
year_dum = pd.get_dummies(df['year'])
work_list = list(work_dum.columns)
year_list = list(year_dum.columns)

df_dum = pd.concat([year_dum, work_dum],axis = 1)
df = pd.concat([df, df_dum],axis = 1)

# Test  
* test용으로는 row1 col3: 'ctrl_exp_turin_co_sample'& discount만 쓸거야

In [85]:
#col2
#dm.ols(stats.api)
#sm.ols로 한번 나온 전적이 있음

df_reg_exp = df[(df['turin_co_sample']==1)&(df['ctrl_exp_turin_co_sample']==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]
#if add
#year = pd.Categorical(df_reg_exp.year)
#id_auth = pd.Categorical(df_reg_exp.id_auth)
#df_reg_exp = df_reg_exp.set_index(['auth_anno','year'])

#df_reg_exp['year'] = year
#df_reg_exp['authority_code'] = id_auth
#exog_var = ['fpsb_auction','year','id_auth','reserve_price', 'municipality']

reg_col = []
for i in work_list:
    reg_col.append(i)
for j in year_list:
    reg_col.append(j)
exog_var = ['fpsb_auction','authority_code','reserve_price','municipality'] + reg_col 
x = df_reg_exp[exog_var]
y = df_reg_exp['discount']
#x = np.asarray(x)

model = sm.OLS(y,x).fit(cov_type='cluster',cov_kwds = {'groups': df_reg_exp['auth_anno']})
result = model.predict(x)
model.summary()
#value is different

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               discount   R-squared:                       0.590
Model:                            OLS   Adj. R-squared:                  0.577
Method:                 Least Squares   F-statistic:                       nan
Date:                Thu, 02 Jul 2020   Prob (F-statistic):                nan
Time:                        00:09:15   Log-Likelihood:                -3834.1
No. Observations:                1262   AIC:                             7748.
Df Residuals:                    1222   BIC:                             7954.
Df Model:                          39                                         
Covariance Type:              cluster                                         
==================================================================================
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
fpsb_auction      11.5732      1.396      8.292      0.000       8.838      14.309
authority_code  2.086e-06   3.87e-07      5.394      0.000    1.33e-06    2.84e-06
reserve_price   1.678e-06   3.12e-07      5.377      0.000    1.07e-06    2.29e-06
municipality      -0.8983      0.626     -1.434      0.152      -2.126       0.330
OG01              -0.8561      0.532     -1.609      0.108      -1.899       0.187
OG02              -0.8443      0.575     -1.469      0.142      -1.971       0.282
OG03              -1.7278      0.537     -3.217      0.001      -2.780      -0.675
OG04              -5.9936      2.902     -2.065      0.039     -11.682      -0.305
OG05           -4.057e-15   5.21e-15     -0.779      0.436   -1.43e-14    6.15e-15
OG06               0.1156      0.595      0.194      0.846      -1.050       1.281
OG07               3.0838      5.278      0.584      0.559      -7.261      13.428
OG08               0.4782      1.582      0.302      0.762      -2.622       3.578
OG09            9.709e-15   1.38e-14      0.703      0.482   -1.74e-14    3.68e-14
OG10               9.1968      0.816     11.274      0.000       7.598      10.796
OG11               3.6730      0.897      4.097      0.000       1.916       5.430
OG12               6.1585      2.155      2.858      0.004       1.935      10.382
OG13            1.207e-14    4.9e-15      2.461      0.014    2.46e-15    2.17e-14
OG18            4.169e-15   1.77e-15      2.356      0.018       7e-16    7.64e-15
OG21            3.075e-15   1.88e-15      1.638      0.101   -6.04e-16    6.75e-15
OG24           -4.074e-15   1.86e-15     -2.188      0.029   -7.72e-15   -4.25e-16
OS01              -5.0225      2.740     -1.833      0.067     -10.392       0.347
OS02               6.4712      0.925      6.999      0.000       4.659       8.283
OS03           -4.399e-15   1.31e-15     -3.345      0.001   -6.98e-15   -1.82e-15
OS04            1.033e-15   1.68e-15      0.616      0.538   -2.25e-15    4.32e-15
OS05               7.8036      1.056      7.391      0.000       5.734       9.873
OS06               3.1012      1.504      2.062      0.039       0.154       6.048
OS07               1.9581      0.628      3.120      0.002       0.728       3.188
OS08               2.8030      3.037      0.923      0.356      -3.149       8.755
OS09           -4.716e-15    1.3e-15     -3.637      0.000   -7.26e-15   -2.17e-15
OS10              13.0574      1.602      8.150      0.000       9.917      16.198
OS11             -10.2344      1.018    -10.055      0.000     -12.229      -8.239
OS12              16.2861      1.235     13.189      0.000      13.866      18.706
OS13           -7.594e-15   8.62e-16     -8.814      0.000   -9.28e-15   -5.91e-15
OS14               2.4518      0.893      2.744      0.006       0.701       4.203
OS18        

In [50]:
#col2 0702 co&ctrl_exp&discount table2(2,1)
#sm.ols version2
#sm.regression
#nan with other outcomes except 'discount'

df_reg_exp = df[(df['turin_co_sample']==1)&(df['ctrl_exp_turin_co_sample']==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]
#if add
#year = pd.Categorical(df_reg_exp.year)
#id_auth = pd.Categorical(df_reg_exp.id_auth)
#df_reg_exp = df_reg_exp.set_index(['auth_anno','year'])

#df_reg_exp['year'] = year
#df_reg_exp['authority_code'] = id_auth
#exog_var = ['fpsb_auction','year','id_auth','reserve_price', 'municipality']

reg_col = []
for i in work_list:
    reg_col.append(i)
for j in year_list:
    reg_col.append(j)
exog_var = ['fpsb_auction','authority_code','reserve_price','municipality'] + reg_col 
x = df_reg_exp[exog_var]
y = df_reg_exp['delay_ratio']
#exog = sm.add_constant(x)
mod = sm.OLS(y,x)
fe_reg = mod.fit(cov_type='cluster',cov_kwds = {'groups': df_reg_exp['auth_anno']})

#result = mod.predict(exog)
print(fe_reg.summary())

#multicollinearity

                            OLS Regression Results                            
Dep. Variable:            delay_ratio   R-squared:                         nan
Model:                            OLS   Adj. R-squared:                    nan
Method:                 Least Squares   F-statistic:                       nan
Date:                Thu, 02 Jul 2020   Prob (F-statistic):                nan
Time:                        11:39:24   Log-Likelihood:                    nan
No. Observations:                1262   AIC:                               nan
Df Residuals:                    1222   BIC:                               nan
Df Model:                          39                                         
Covariance Type:              cluster                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
fpsb_auction          nan        nan        

C:\Users\thxkn\anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
C:\Users\thxkn\anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
C:\Users\thxkn\anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)
C:\Users\thxkn\anaconda3\lib\site-packages\scipy\stats\stats.py:1548: RuntimeWarning: invalid value encountered in sign
  term2 = np.sign(denom) * np.where(denom == 0.0, np.nan,


In [19]:
# mutlicollinearity
from statsmodels.stats.outliers_influence import variance_inflation_factor

def calc_vif(X):

    # Calculating VIF
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

    return(vif)

In [25]:
#check multicollinearity
#will be problematic if VIF >=10
#exog_var = exog_var = ['fpsb_auction','id_auth','reserve_price','municipality'] + reg_col 
X = df_reg_exp.loc[:,exog_var]
vif = calc_vif(X)

C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1685: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss


In [83]:
#check multicollinearity
#will be problematic if VIF >=10
#exog_var = exog_var = ['fpsb_auction','authority_code','reserve_price','municipality'] + reg_col
#authority code로 넣어야 authority and municipality 의 mutilcolinearity없어짐
#아냐, multicolinearity는 없어져야 했음 id_auth를 넣고 multicolinearity 때문에 municipality를 삭제하면 됨

reg_col = []
for i in work_list:
    reg_col.append(i)
for j in year_list:
    reg_col.append(j)
exog_var = ['overrun_ratio','fpsb_auction','id_auth','reserve_price'] + reg_col 


X = df_reg_exp.loc[:,exog_var]
vif = calc_vif(X)
print(vif)

MissingDataError: exog contains inf or nans

In [69]:
#col2
#다중공선성 or NaN 미리 제거
for i in range(len(vif)):
    if np.isnan(vif.loc[i, 'VIF']) == True:
        col_name = vif.loc[i, 'variables']
        print(col_name)
        reg_col.remove(col_name)

print(reg_col)

OG05
OG09
OG13
OG18
OG21
OG24
OS03
OS04
OS09
OS13
OS32
OS33
2007.0
2008.0
2009.0
2010.0
['OG01', 'OG02', 'OG03', 'OG04', 'OG06', 'OG07', 'OG08', 'OG10', 'OG11', 'OG12', 'OS01', 'OS02', 'OS05', 'OS06', 'OS07', 'OS08', 'OS10', 'OS11', 'OS12', 'OS14', 'OS18', 'OS19', 'OS21', 'OS22', 'OS23', 'OS24', 'OS26', 'OS28', 'OS30', 'OS34', 2000.0, 2001.0, 2002.0, 2003.0, 2004.0, 2005.0, 2006.0]


In [70]:
#col2 0702 co&ctrl_exp&discount table2(2,1)
#sm.ols version3
#sm.regression
#prbm: nan with other outcomes except 'discount'
#trial : multicollinearity
#includes authority_code and deleted nan dummies

df_reg_exp = df[(df['turin_co_sample']==1)&(df['ctrl_exp_turin_co_sample']==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]
#if add
#year = pd.Categorical(df_reg_exp.year)
#id_auth = pd.Categorical(df_reg_exp.id_auth)
#df_reg_exp = df_reg_exp.set_index(['auth_anno','year'])

#df_reg_exp['year'] = year
#df_reg_exp['authority_code'] = id_auth
#exog_var = ['fpsb_auction','year','id_auth','reserve_price', 'municipality']

#reg_col = []
#for i in work_list:
#    reg_col.append(i)
#for j in year_list:
#    reg_col.append(j)
exog_var = ['fpsb_auction','id_auth','reserve_price'] + reg_col 
x = df_reg_exp[exog_var]
y = df_reg_exp['delay_ratio']
#exog = sm.add_constant(x)
mod = sm.OLS(y,x, hasconst = True)
fe_reg = mod.fit(cov_type='cluster',cov_kwds = {'groups': df_reg_exp['auth_anno']})

#result = mod.predict(exog)
print(fe_reg.summary())

#multicollinearity

                            OLS Regression Results                            
Dep. Variable:            delay_ratio   R-squared:                         nan
Model:                            OLS   Adj. R-squared:                    nan
Method:                 Least Squares   F-statistic:                       nan
Date:                Thu, 02 Jul 2020   Prob (F-statistic):                nan
Time:                        11:48:47   Log-Likelihood:                    nan
No. Observations:                1262   AIC:                               nan
Df Residuals:                    1223   BIC:                               nan
Df Model:                          38                                         
Covariance Type:              cluster                                         
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
fpsb_auction         nan        nan        nan

C:\Users\thxkn\anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
C:\Users\thxkn\anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
C:\Users\thxkn\anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)
C:\Users\thxkn\anaconda3\lib\site-packages\scipy\stats\stats.py:1548: RuntimeWarning: invalid value encountered in sign
  term2 = np.sign(denom) * np.where(denom == 0.0, np.nan,


In [71]:
#col2
#Panel OLS
#discount
#doesnot have full rank: set index 잘 만져서 해결했었음


df_reg_exp = df[(df['turin_co_sample']==1)&(df['ctrl_exp_turin_co_sample']==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]
id_auth = pd.Categorical(df_reg_exp.id_auth)
#work_category = pd.Categorical(df_reg_exp.work_category)
year = pd.Categorical(df_reg_exp.year)

df_reg_exp = df_reg_exp.set_index(['auth_anno','year'])
df_reg_exp['year'] = year
df_reg_exp['authority_code'] = id_auth
#df_reg_exp['work_category'] = work_category


#Iwork_category = []
#for i in range(len(work_ca_order)):
#    Iwork_category.append('Iwork_ca_'+str(i+1))
#reg_col = []
#for i in work_list:
#    reg_col.append(i)
#for j in year_list:
#    reg_col.append(j)
exog_var = ['fpsb_auction','id_auth','year','reserve_price'] + reg_col 
#exog_var = np.array(exog_var)
exog = sm.add_constant(df_reg_exp[exog_var])
mod = PanelOLS(df_reg_exp.discount, exog, time_effects=True, drop_absorbed = True)  
fe_res = mod.fit()
print(fe_res)

#exog does not have full column rank 문제에서 계속 막힘

ValueError: exog does not have full column rank.

In [72]:
#mt.reg
#singular matrix error 너무 걸려서 못 씀
import econtools as econ
import econtools.metrics as mt

df_reg_exp = df[(df['turin_co_sample']==1)&(df['ctrl_exp_turin_co_sample']==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]
mt.reg(df_reg_exp,'delay_ratio',['fpsb_auction','reserve_price'] + reg_col, fe_name = 'authority_code', cluster = 'auth_anno')

C:\Users\thxkn\anaconda3\lib\site-packages\econtools\metrics\core.py:203: RuntimeWarning: invalid value encountered in sqrt
  se = pd.Series(np.sqrt(np.diagonal(vce)), index=vce.columns)
C:\Users\thxkn\anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1827: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= np.asarray(_b)) & cond0


Dependent variable:	delay_ratio
N:			1110
R-squared:		0.1216
Estimation method:	OLS
VCE method:		Cluster
  Cluster variable:	  auth_anno
  No. of clusters:	  99
Fixed effects by:	authority_code
  No. of FE:		  15
                             coeff                  se      t   p>t                CI_low             CI_high
fpsb_auction                35.300              11.959  2.952 0.004                11.569              59.032
reserve_price               -0.000               0.000 -2.008 0.047                -0.000              -0.000
OG01          -580478055792879.000 587425575418275.250 -0.988 0.326 -1746204992108318.500 585248880522560.500
OG02          -580478055792879.000 587425575418268.500 -0.988 0.326 -1746204992108305.000 585248880522547.000
OG03          -580478055792872.000 587425575418274.375 -0.988 0.326 -1746204992108309.750 585248880522565.750
OG04          -580478055792900.000 587425575418266.000 -0.988 0.326 -1746204992108321.250 585248880522521.250
OG06          -58

# Col2: value는 달라도 그냥 고/ sm.OLS

In [93]:
#indexing 하는 순간 에러남
#outcome = ['discount', 'delay_ratio', 'overrun_ratio', 'days_to_award']

df_reg_exp = df[(df['turin_co_sample']==1)&(df['ctrl_exp_turin_co_sample']==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]
reg_col = []
for i in work_list:
    reg_col.append(i)
for j in year_list:
    reg_col.append(j)
exog_var = ['fpsb_auction','authority_code','reserve_price','municipality'] + reg_col 
x = df_reg_exp[exog_var]

y = df_reg_exp['discount']
model = sm.OLS(y,x).fit(cov_type='cluster',cov_kwds = {'groups': df_reg_exp['auth_anno']})
result = model.predict(x)
model.summary()
#value diff
#문제점 1: omitted되어야 할 게 omitted되지 않음
#문제점 2: outcome 바꾸면 nan값으로만 나옴: r_matrix performs f_test for using dimensions that are asymptotically non-normal

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               discount   R-squared:                       0.590
Model:                            OLS   Adj. R-squared:                  0.577
Method:                 Least Squares   F-statistic:                       nan
Date:                Thu, 02 Jul 2020   Prob (F-statistic):                nan
Time:                        00:16:29   Log-Likelihood:                -3834.1
No. Observations:                1262   AIC:                             7748.
Df Residuals:                    1222   BIC:                             7954.
Df Model:                          39                                         
Covariance Type:              cluster                                         
==================================================================================
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
fpsb_auction      11.5732      1.396      8.292      0.000       8.838      14.309
authority_code  2.086e-06   3.87e-07      5.394      0.000    1.33e-06    2.84e-06
reserve_price   1.678e-06   3.12e-07      5.377      0.000    1.07e-06    2.29e-06
municipality      -0.8983      0.626     -1.434      0.152      -2.126       0.330
OG01              -0.8561      0.532     -1.609      0.108      -1.899       0.187
OG02              -0.8443      0.575     -1.469      0.142      -1.971       0.282
OG03              -1.7278      0.537     -3.217      0.001      -2.780      -0.675
OG04              -5.9936      2.902     -2.065      0.039     -11.682      -0.305
OG05           -4.057e-15   5.21e-15     -0.779      0.436   -1.43e-14    6.15e-15
OG06               0.1156      0.595      0.194      0.846      -1.050       1.281
OG07               3.0838      5.278      0.584      0.559      -7.261      13.428
OG08               0.4782      1.582      0.302      0.762      -2.622       3.578
OG09            9.709e-15   1.38e-14      0.703      0.482   -1.74e-14    3.68e-14
OG10               9.1968      0.816     11.274      0.000       7.598      10.796
OG11               3.6730      0.897      4.097      0.000       1.916       5.430
OG12               6.1585      2.155      2.858      0.004       1.935      10.382
OG13            1.207e-14    4.9e-15      2.461      0.014    2.46e-15    2.17e-14
OG18            4.169e-15   1.77e-15      2.356      0.018       7e-16    7.64e-15
OG21            3.075e-15   1.88e-15      1.638      0.101   -6.04e-16    6.75e-15
OG24           -4.074e-15   1.86e-15     -2.188      0.029   -7.72e-15   -4.25e-16
OS01              -5.0225      2.740     -1.833      0.067     -10.392       0.347
OS02               6.4712      0.925      6.999      0.000       4.659       8.283
OS03           -4.399e-15   1.31e-15     -3.345      0.001   -6.98e-15   -1.82e-15
OS04            1.033e-15   1.68e-15      0.616      0.538   -2.25e-15    4.32e-15
OS05               7.8036      1.056      7.391      0.000       5.734       9.873
OS06               3.1012      1.504      2.062      0.039       0.154       6.048
OS07               1.9581      0.628      3.120      0.002       0.728       3.188
OS08               2.8030      3.037      0.923      0.356      -3.149       8.755
OS09           -4.716e-15    1.3e-15     -3.637      0.000   -7.26e-15   -2.17e-15
OS10              13.0574      1.602      8.150      0.000       9.917      16.198
OS11             -10.2344      1.018    -10.055      0.000     -12.229      -8.239
OS12              16.2861      1.235     13.189      0.000      13.866      18.706
OS13           -7.594e-15   8.62e-16     -8.814      0.000   -9.28e-15   -5.91e-15
OS14               2.4518      0.893      2.744      0.006       0.701       4.203
OS18        

In [ ]:
#vif한 후에 돌리기

y = df_reg_exp['discount']
model = sm.OLS(y,x,missing = 'drop').fit(cov_type='cluster',cov_kwds = {'groups': df_reg_exp['auth_anno']})
#result = model.predict(x)
print(model.summary())

In [76]:
import econtools as econ
import econtools.metrics as mt

df_reg_exp = df[(df['turin_co_sample']==1)&(df['ctrl_exp_turin_co_sample']==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]

# Cal2 = multicolinearity check and then delete NaN or vif > 10

In [ ]:
#col2 setting
df_reg_exp = df[(df['turin_co_sample']==1)&(df['ctrl_exp_turin_co_sample']==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]

In [79]:
# mutlicollinearity function
from statsmodels.stats.outliers_influence import variance_inflation_factor
def calc_vif(X):

    # Calculating VIF
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

    return(vif)

#vif cal
#first, make a column list
reg_col = []
for i in work_list:
    reg_col.append(i)
for j in year_list:
    reg_col.append(j)
exog_var = ['fpsb_auction','id_auth','reserve_price','municipality'] + reg_col 


#check multicollinearity
X = df_reg_exp.loc[:,exog_var]
vif = calc_vif(X)
print(vif)


#delete from col list
for i in range(len(vif)):
    if np.isnan(vif.loc[i, 'VIF']) == True:
        exog_var.remove(vif.loc[i, 'variables'])
    elif vif.loc[i,'VIF'] > 10 &  vif.loc[i,'variables'] != 'fpsb_auction'& vif.loc[i,'variables'] != 'id_auth':
        exog_var.remove(vif.loc[i,'variables'])

print(exog_var)

C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1685: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss


         variables       VIF
0     fpsb_auction  2.255944
1   authority_code  2.324504
2    reserve_price  1.122674
3     municipality  1.248414
4             OG01       inf
5             OG02       inf
6             OG03       inf
7             OG04       inf
8             OG05       NaN
9             OG06       inf
10            OG07       inf
11            OG08       inf
12            OG09       NaN
13            OG10       inf
14            OG11       inf
15            OG12       inf
16            OG13       NaN
17            OG18       NaN
18            OG21       NaN
19            OG24       NaN
20            OS01       inf
21            OS02       inf
22            OS03       NaN
23            OS04       NaN
24            OS05       inf
25            OS06       inf
26            OS07       inf
27            OS08       inf
28            OS09       NaN
29            OS10       inf
30            OS11       inf
31            OS12       inf
32            OS13       NaN
33            

"for i in range(len(vif)):\n    if np.isnan(vif.loc[i, 'VIF']) == True:\n        reg_col.remove(vif.loc[i, 'variables'])\n    elif vif.loc[i,'VIF'] > 10 &  vif.loc[i,'variables'] != 'fpsb_auction'& vif.loc[i,'variables'] != 'authority_code':\n        reg_col.remove(vif.loc[i,'variables'])\n\nprint(reg_col)"

In [ ]:
#winning discount
mt.reg(df_reg_exp,'discount', reg_col, fe_name = 'authority_code', cluster = 'auth_anno')

In [ ]:
#delay_ratio
mt.reg(df_reg_exp,'delay_ratio', reg_col, fe_name = 'authority_code', cluster = 'auth_anno')

In [ ]:
#overrun_ratio (problem)
#nan overrun ratio
#안먹히면 OLS nan 값 제거하고 넣기?
mt.reg(df_reg_exp,'overrun_ratio', reg_col, fe_name = 'authority_code', cluster = 'auth_anno')